In [52]:
import pandas as pd

## Load data

In [53]:
train = pd.read_csv('../Dataset/vihsd/train.csv')

In [54]:
train.head()

,free_text,label_id
0,Em được làm fan cứng luôn rồi nè ❤️ reaction q...,0
1,Đúng là bọn mắt híp lò xo thụt :))) bên việt n...,2
2,Đậu Văn Cường giờ giống thằng sida hơn à,0
3,CÔN ĐỒ CỤC SÚC VÔ NHÂN TÍNH ĐỀ NGHI VN. NHÀ NƯ...,2
4,Từ lý thuyết đến thực hành là cả 1 câu chuyện ...,0


In [55]:
print(train['label_id'].value_counts())

label_id
0    19886
2     2556
1     1606
Name: count, dtype: int64


## Remove null values

In [56]:
train.isnull().sum()

free_text    2
label_id     0
dtype: int64

In [57]:
train = train.dropna()

## Remove #ERROR! values

In [58]:
error_rows = train[train['free_text'].str.contains("#ERROR!")]
print(error_rows)

      free_text  label_id
1729    #ERROR!         0
1867    #ERROR!         0
3568    #ERROR!         0
10788   #ERROR!         0
11218   #ERROR!         0
11674   #ERROR!         0
15413   #ERROR!         0
16080   #ERROR!         0
18660   #ERROR!         2
20848   #ERROR!         1


In [59]:
train = train[~train['free_text'].str.contains("#ERROR!")]

## Remove duplicates

In [60]:
duplicate_rows = train[train['free_text'].duplicated()]
print(duplicate_rows)

                                        free_text  label_id
225                              reaction thầy ơi         0
442                    đặt khô gà ở đâu v thầy ơi         0
466                              Mệt quá thầy ạ 😞         0
696                                         xin ạ         0
790                     Tại sao phải trả lời thầy         0
...                                           ...       ...
23986  Đình Quang ụa đâu có giải gì bên vn đau ta         0
23992                        Thành Huy Hoàng Tiến         0
24008                     Tự hào thầy Ba gold :))         0
24010                          Lỡ tay thầy ơi :))         0
24022                        Phong Chau cười ỉa 😂         0

[1480 rows x 2 columns]


In [61]:
# Drop duplicates, keep first occurence, make changes directly to the dataframe
train.drop_duplicates(subset='free_text', keep='first', inplace=True)

## Remove outliers
- Some examples have a very big length of characters, not containing helpful information. We will remove them.

In [62]:
num_long_rows = (train['free_text'].str.len() > 500).sum()
print(num_long_rows)

14


In [63]:
train = train[train['free_text'].str.len() <= 500]

## Remove special characters

### URL

In [64]:
# Create a boolean mask for rows that contain a URL
mask = train['free_text'].str.contains(r'http\S+|www\S+', regex=True)

# Use the mask to select a subset of the DataFrame
url_examples = train[mask]

# Print the first few examples
print(url_examples.head())

                                              free_text  label_id
1222  Xem ngay hậu trường cực hiếm của cô Minh Hiếu ...         0
1432  FB chính LinDa: https://www.facebook.com/linda...         0
1839  https://youtu.be/tvyO2B3oEYk thư giãn đã cả nh...         0
2020     @công danh nguyen https://youtu.be/fSypgwW1L_s         0
6891  biệt thự của nude tiger: https://www.google.co...         0


In [65]:
# URL removal
train['free_text'] = train['free_text'].str.replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)

### Punctuation

In [66]:
train['free_text'] = train['free_text'].str.replace(r'[^\w\s]', '', regex=True)

### Stopwords

In [67]:
# Load stopwords
with open('./Stopwords/vietnamese-stopwords.txt', 'r') as f:
    stopwords = f.read().splitlines()

# Function to remove stopwords

def remove_stopwords(text):
    words = text.split()
    words = [word for word in words if word not in stopwords]
    return ' '.join(words)


# Remove stopwords from 'free_text'
train['free_text'] = train['free_text'].apply(remove_stopwords)

## Save cleaned data

In [68]:
train.to_csv('../Dataset_Cleaned/clean_train_vihsd.csv', index=False)